In [6]:
import pandas as pd
import requests
import os

def fetch_data(source, is_local=True):
    """
    Fetches data from a local file or remote URL.
    
    Parameters:
        source (str): File path or URL to the data source.
        is_local (bool): True if fetching from a local file, False for a URL.
    
    Returns:
        pd.DataFrame: The data loaded into a DataFrame.
    """
    if is_local:
        # Read from a local file
        if os.path.isfile(source):
            if source.endswith('.csv'):
                return pd.read_csv(source)
            elif source.endswith('.json'):
                return pd.read_json(source)
            else:
                raise ValueError("Unsupported file format. Only CSV and JSON are supported.")
        else:
            raise FileNotFoundError("File not found at the specified path.")
    else:
        # Download from a URL
        response = requests.get(source)
        if response.status_code == 200:
            if source.endswith('.csv'):
                # Read the content into a DataFrame
                return pd.read_csv(pd.compat.StringIO(response.text))
            elif source.endswith('.json'):
                # Read the content into a DataFrame
                return pd.read_json(response.content)
            else:
                raise ValueError("Unsupported URL format. Only CSV and JSON are supported.")
        else:
            raise ConnectionError(f"Failed to fetch data from URL. Status code: {response.status_code}")
